In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import csv
import numpy as np
import re
import pandas as pd
import pyvi
from pyvi import ViTokenizer, ViPosTagger

In [3]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
file_path = '/content/gdrive/MyDrive/IE403/sentiments_raw.txt'

In [5]:
with open("/content/gdrive/MyDrive/IE403/Stopword.txt",'rb') as f:
    contents = f.read()
contents = contents.decode("utf-16")
contents = contents.split("\r\n")
stopword = set(contents)
 
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

In [6]:
def apply_teencode_transformations(sentence):
    code = ['ctrai', 'khôg', 'bme', 'cta', 'mih', 'mqh', 'cgai', 'nhữg', 'mng', 'svtn', 'r', 'qtam', 'thươg', 'qtâm', 'chug', 'trườg', 'thoy', 'đki', 'atsm', 'ạk', 'cv', 'vch', 'cùg', 'pn', 'pjt', 'thjk', 'keke', 'ktra', 'nek', 'cgái', 'nthe', 'chúg', 'kái', 'tìh', 'phòg', 'lòg', 'từg', 'rằg', 'sốg', 'thuj', 'thuơng', 'càg', 'đky', 'bằg', 'sviên', 'ák', 'đág', 'nvay', 'nhjeu', 'xg', 'zồi', 'trag', 'zữ', 'atrai', 'kte', 'độg', 'lmht', 'gắg', 'đzai', 'thgian', 'plz', 'đồg', 'btrai', 'nthê', 'hìhì', 'vọg', 'hihe', 'đôg', 'răg', 'thườg', 'tcảm', 'đứg', 'ksao', 'dz', 'hjxhjx', 'cmày', 'xuốg', 'nkư', 'lquan', 'tiếg', 'hajz', 'xih', 'hìh', 'thàh', 'ngke', 'dzậy', 'teencode', 'tnào', 'tưởg', 'ctrinh', 'phog', 'hôg', 'zìa', 'kũg', 'ntnao', 'trọg', 'nthế', 'năg', 'ngđó', 'lquen', 'riêg', 'ngag', 'hêhê', 'bnhiu', 'ngốk', 'kậu', 'highland', 'kqua', 'htrc', 'địh', 'gđình', 'giốg', 'csống', 'xug', 'zùi', 'bnhiêu', 'cbị', 'kòn', 'buôg', 'csong', 'chàg', 'chăg', 'ngàh', 'llac', 'nkưng', 'nắg', 'tíh', 'khoảg', 'thík', 'ngđo', 'ngkhác', 'thẳg', 'kảm', 'dàh', 'júp', 'lặg', 'vđê', 'bbè', 'bóg', 'dky', 'dòg', 'uốg', 'tyêu', 'snvv', 'đthoại', 'qhe', 'cviec', 'tượg', 'qà', 'thjc', 'nhưq', 'cđời', 'bthường', 'zà', 'đáh', 'xloi', 'zám', 'qtrọng', 'bìh', 'lzi', 'qhệ', 'đhbkhn', 'hajzz', 'kủa', 'lz', 'đhkhtn', 'đóg', 'cka', 'lgi', 'nvậy', 'qả', 'đkiện', 'nèk', 'tlai', 'bsĩ', 'hkì', 'đcsvn', 'vde', 'chta', 'òy', 'ltinh', 'ngyeu', 'đthoai', 'snghĩ', 'nặg', 'họk', 'dừg', 'hphúc', 'hiha', 'wtâm', 'thíck', 'chuện', 'lạh', 'fây', 'ntnày', 'lúk', 'haj', 'ngía', 'mớj', 'hsơ', 'ctraj', 'nyêu', 'điiiiiii', 'rồii', 'c', 'kih', 'kb', 'hixxx', 'dthương', 'nhiềuuu', 'ctrình', 'mìnk', 'mjh', 'ng', 'vc', 'uhm', 'thỳ', 'nyc', 'tks', 'nàg', 'thôii', 'đjên', 'bgái', 'vớii', 'xink', 'hđộng', 'đhọc', 'mk', 'bn', 'thik', 'cj', 'mn', 'nguoi', 'nógn', 'hok', 'ko', 'bik', 'vs', 'cx', 'mik', 'wtf', 'đc', 'cmt', 'ck', 'chk', 'ngta', 'gđ', 'oh', 'vk', 'ctác', 'sg', 'ae', 'ah', 'ạh', 'rì', 'ms', 'vn', 'nhaa', 'cũg', 'đag', 'ơiii', 'hic', 'ace', 'àk', 'uh', 'cmm', 'cmnr', 'ơiiii', 'hnay', 'ukm', 'tq', 'ctr', 'đii', 'nch', 'trieu', 'hahah', 'nta', 'ngèo', 'kêh', 'ak', 'ad', 'j', 'ny', 'dc', 'qc', 'baoh', 'zui', 'zẻ', 'tym', 'aye', 'eya', 'fb', 'insta', 'z', 'thich', 'vcl', 'đt', 'acc', 'lol', 'loz', 'lozz', 'trc', 'chs', 'đhs', 'qá', 'ntn', 'wá', 'zậy', 'zô', 'ytb', 'vđ', 'vchg', 'sml', 'xl', 'cmn', 'face', 'hjhj', 'vv', 'ns', 'iu', 'vcđ', 'in4', 'qq', 'sub', 'kh', 'zạ', 'oy', 'jo', 'clmm', 'bsvv', 'troai', 'wa', 'hjx', 'e', 'ik', 'ji', 'ce', 'lm', 'đz', 'sr', 'ib', 'hoy', 'đbh', 'k', 'vd', 'a', 'cũng z', 'z là', 'unf', 'my fen', 'fen', 'cty', 'on lai', 'u hai ba', 'kô', 'đtqg', 'hqua', 'xog', 'uh', 'uk', 'nhoé', 'biet', 'quí', 'stk', 'hong kong', 'đươc', 'nghành', 'nvqs', 'ngừoi', 'trog', 'tgian', 'biêt', 'fải', 'nguời', 'tđn', 'bth', 'vcđ', 'tgdd', 'khg', 'nhưg', 'thpt', 'thằg', 'đuợc', 'dc', 'đc', 'ah', 'àh', 'ku', 'thým', 'onl', 'zô', 'zú', 'cmnd', 'sđt', 'klq']
    chuanhoa = ['con trai', 'không', 'bố mẹ', 'chúng ta', 'mình', 'mối quan hệ', 'con gái', 'những', 'mọi người', 'sinh viên tình nguyện', 'rồi', 'quan tâm', 'thương', 'quan tâm', 'chung', 'trường', 'thôi', 'đăng ký', 'ảo tưởng sức mạnh', 'ạ', 'công việc', 'vãi chưởng', 'cùng', 'bạn', 'biết', 'thích', 'ce ce', 'kiểm tra', 'nè', 'con gái', 'như thế', 'chúng', 'cái', 'tình', 'phòng', 'lòng', 'từng', 'rằng', 'sống', 'thôi', 'thương', 'càng', 'đăng ký', 'bằng', 'sinh viên', 'á', 'đáng', 'như vậy', 'nhiều', 'xuống', 'rồi', 'trang', 'dữ', 'anh trai', 'kinh tế', 'động', 'liên minh huyền thoại', 'gắng', 'đẹp trai', 'thời gian', 'pờ ly', 'đồng', 'bạn trai', 'như thế', 'hì hì', 'vọng', 'hi he', 'đông', 'răng', 'thường', 'tình cảm', 'đứng', 'không sao', 'đẹp trai', 'hix hix', 'chúng mày', 'xuống', 'như', 'liên quan', 'tiếng', 'hai', 'xinh', 'hình', 'thành', 'nghe', 'dậy', 'tin cốt', 'thế nào', 'tưởng', 'chương trình', 'phong', 'không', 'gì', 'cũng', 'như thế nào', 'trọng', 'như thế', 'năng', 'người đó', 'làm quen', 'riêng', 'ngang', 'hê hê', 'bao nhiêu', 'ngốc', 'cậu', 'hai lừn', 'kết quả', 'hôm trước', 'định', 'gia đinh', 'giống', 'cuộc sống', 'xùng', 'rồi', 'bao nhiêu', 'chuẩn bị', 'còn', 'buông', 'cuộc sống', 'chàng', 'chăng', 'ngành', 'liên lạc', 'nhưng', 'nắng', 'tính', 'khoảng', 'thích', 'người đó', 'người khác', 'thẳng', 'cảm', 'dành', 'giúp', 'lặng', 'vấn đề', 'bạn bè', 'bóng', 'đăng ký', 'dòng', 'uống', 'tình yêu', 'sinh nhật vui vẻ', 'điện thoại', 'quan hệ', 'công việc', 'tượng', 'quà', 'thích', 'nhưng', 'cuộc đời', 'bình thường', 'già', 'đánh', 'xin lỗi', 'dám', 'quan trọng', 'bình', 'làm gì', 'quan hệ', 'đại học bách khoa hà nội', 'hai', 'của', 'làm gì', 'đại học khoa học tự nhiên', 'đóng', 'cha', 'làm gì', 'như vậy', 'quả', 'điều kiện', 'nè', 'tương lai', 'bác sĩ', 'học kỳ', 'đảng cộng sản việt nam', 'vấn đề', 'chúng ta', 'rồi', 'linh tinh', 'người yêu', 'điện thoại', 'suy nghĩ', 'nặng', 'học', 'dừng', 'hạnh phúc', 'hi ha', 'quan tâm', 'thích', 'chuyện', 'lạnh', 'phây', 'như thế này', 'lúc', 'hai', 'nghía', 'mới', 'hồ sơ', 'con trai', 'người yêu', 'đi', 'rồi', 'chị', 'kinh', 'kết bạn', 'hích', 'dễ thương', 'nhiều', 'chương trình', 'mình', 'mình', 'người', 'vợ chồng', 'ừm', 'thì', 'người yêu cũ', 'thanks', 'nàng', 'thôi', 'điên', 'bạn gái', 'với', 'xinh', 'hành động', 'đại học', 'mình', 'bạn', 'thích', 'chị', 'mọi người', 'người', 'nóng', 'không', 'không', 'biết', 'với', 'cũng', 'mình', 'what the fuck', 'được', 'comment', 'chồng', 'chồng', 'người ta', 'gia đình', 'ồ', 'vợ', 'công tác', 'sài gòn', 'anh em', 'à', 'ạ', 'gì', 'mới', 'việt nam', 'nha', 'cũng', 'đang', 'ơi', 'hích', 'anh chị em', 'à', 'ừ', 'con mẹ mày', 'con mẹ nó rồi', 'ơi', 'hôm nay', 'ừm', 'trung quốc', 'chương trình', 'đi', 'nói chuyện', 'triệu', 'ha ha', 'người ta', 'nghèo', 'kênh', 'à', 'admin', 'gì', 'người yêu', 'được', 'quảng cáo', 'bao giờ', 'vui', 'vẻ', 'tim', 'anh yêu em', 'em yêu anh', 'facebook', 'instagram', 'vậy', 'thích', 'vờ cờ lờ', 'điện thoại', 'account', 'lồn', 'lồn', 'lồn', 'trước', 'chẳng hiểu sao', 'đéo hiểu sao', 'quá', 'như thế nào', 'quá', 'vậy', 'vô', 'youtube', 'vãi đái', 'vãi chưởng', 'sấp mặt lờ', 'xin lỗi', 'con mẹ nó', 'facebook', 'hi hi', 'vui vẻ', 'nói', 'yêu', 'vãi cải đái', 'info', 'quằn què', 'subcribe', 'không', 'vậy', 'rồi', 'giờ', 'cái lồn mẹ mày', 'buổi sáng vui vẻ', 'trai', 'quá', 'hix', 'em', 'ý', 'gì', 'chị em', 'làm', 'đẹp giai', 'sorry', 'inbox', 'thôi', 'đéo bao giờ', 'không', 'ví dụ', 'anh', 'cũng vậy', 'vậy là', 'unfriend', 'my friend', 'friend', 'công ty', 'online', 'u23', 'không', 'đội tuyển quốc gia', 'hôm qua', 'xong', 'ừ', 'ừ', 'nhé', 'biết', 'quý', 'số tài khoản', 'hồng kông', 'được', 'ngành', 'nghĩa vụ quân sự', 'người', 'trong', 'thời gian', 'biết', 'phải', 'người', 'thế đéo nào', 'bình thường', 'vãi cả đái', 'thế giới di động', 'không', 'nhưng', 'trung học phổ thông', 'thằng', 'được', 'được', 'được', 'à', 'à', 'cu', 'thím', 'online', 'dô', 'vú', 'chứng minh nhân dân', 'số điện thoại', 'không liên quan']
    data = {"code" : code, "chuanhoa" : chuanhoa}
    teencode = pd.DataFrame(data)
    result = [x.strip() for x in sentence.split()]
    for i in range(0, len(result)):
        for j in range(0, len(teencode)):
            if (result[i] == teencode.at[j, "code"]):
                result[i] = teencode.at[j, "chuanhoa"]
    x = " ".join(result)
    x.strip()
    return x

In [7]:
file_path1 = file_path

In [8]:
def preprocess_text_file(file_path):
    normalized_lines = []

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')

        for row in reader:
            # Kiểm tra số lượng cột hợp lệ
            if len(row) != 5:
                print(f"Ignored line due to invalid number of columns: {row}")
                continue

            comment = row[4]  # Lấy nội dung trong cột "comment"

            # Loại bỏ các hàng chứa từ có 8 chữ cái trở lên
            words = comment.split()
            if any(len(word) >= 8 for word in words):
                continue

            # Chuẩn hóa chữ hoa thành chữ thường
            normalized_comment = comment.lower()

            row[4] = normalized_comment  # Cập nhật nội dung đã chuẩn hóa vào cột "comment"

            normalized_line = '\t'.join(row)  # Ghép các cột thành dòng đã chuẩn hóa

            normalized_lines.append(normalized_line)  # Thêm dòng đã chuẩn hóa vào danh sách

    # Xuất kết quả ra file csv
    csv_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.csv'  # Đường dẫn đến file csv
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows([line.split('\t') for line in normalized_lines])

    # Xuất kết quả ra file txt
    txt_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.txt'  # Đường dẫn đến file txt
    np.savetxt(txt_file_path, normalized_lines, fmt='%s', delimiter='\t')

# Ví dụ sử dụng hàm preprocess_text_file với một file có cột "comment"

preprocess_text_file(file_path1)


In [9]:
file_path1 = '/content/gdrive/MyDrive/IE403/data_normalized.txt'
def preprocess_text_file(file_path):
    normalized_lines = []

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')

        for row in reader:
            # Kiểm tra số lượng cột hợp lệ
            if len(row) != 5:
                print(f"Ignored line due to invalid number of columns: {row}")
                continue

            comment = row[4]  # Lấy nội dung trong cột "comment"

            # Xử lý clear teencode transformation
            normalized_comment = apply_teencode_transformations(comment)
            
            row[4] = normalized_comment  # Cập nhật nội dung đã chuẩn hóa vào cột "comment"

            normalized_line = '\t'.join(row)  # Ghép các cột thành dòng đã chuẩn hóa

            normalized_lines.append(normalized_line)  # Thêm dòng đã chuẩn hóa vào danh sách

    # Xuất kết quả ra file csv
    csv_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.csv'  # Đường dẫn đến file csv
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows([line.split('\t') for line in normalized_lines])

    # Xuất kết quả ra file txt
    txt_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.txt'  # Đường dẫn đến file txt
    np.savetxt(txt_file_path, normalized_lines, fmt='%s', delimiter='\t')

# Ví dụ sử dụng hàm preprocess_text_file với một file có cột "comment"
preprocess_text_file(file_path1)

In [10]:
file_path1 = '/content/gdrive/MyDrive/IE403/data_normalized.txt'
def preprocess_text_file(file_path):
    normalized_lines = []

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')

        for row in reader:
            # Kiểm tra số lượng cột hợp lệ
            if len(row) != 5:
                print(f"Ignored line due to invalid number of columns: {row}")
                continue

            comment = row[4]  # Lấy nội dung trong cột "comment"

            # Loại bỏ stop word trong tiếng Việt
            normalized_comment = remove_stopwords(comment)

            row[4] = normalized_comment  # Cập nhật nội dung đã chuẩn hóa vào cột "comment"

            normalized_line = '\t'.join(row)  # Ghép các cột thành dòng đã chuẩn hóa

            normalized_lines.append(normalized_line)  # Thêm dòng đã chuẩn hóa vào danh sách

    # Xuất kết quả ra file csv
    csv_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.csv'  # Đường dẫn đến file csv
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows([line.split('\t') for line in normalized_lines])

    # Xuất kết quả ra file txt
    txt_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.txt'  # Đường dẫn đến file txt
    np.savetxt(txt_file_path, normalized_lines, fmt='%s', delimiter='\t')

# Ví dụ sử dụng hàm preprocess_text_file với một file có cột "comment"
preprocess_text_file(file_path1)

In [11]:
file_path1 = '/content/gdrive/MyDrive/IE403/data_normalized.txt'
def preprocess_text_file(file_path):
    normalized_lines = []

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')

        for row in reader:
            # Kiểm tra số lượng cột hợp lệ
            if len(row) != 5:
                print(f"Ignored line due to invalid number of columns: {row}")
                continue

            comment = row[4]  # Lấy nội dung trong cột "comment"

            # Xử lý Word segmentation trong tiếng Việt
            normalized_comment = ViTokenizer.tokenize(comment)

            row[4] = normalized_comment  # Cập nhật nội dung đã chuẩn hóa vào cột "comment"

            normalized_line = '\t'.join(row)  # Ghép các cột thành dòng đã chuẩn hóa

            normalized_lines.append(normalized_line)  # Thêm dòng đã chuẩn hóa vào danh sách

    # Xuất kết quả ra file csv
    csv_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.csv'  # Đường dẫn đến file csv
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows([line.split('\t') for line in normalized_lines])

    # Xuất kết quả ra file txt
    txt_file_path = '/content/gdrive/MyDrive/IE403/data_normalized.txt'  # Đường dẫn đến file txt
    np.savetxt(txt_file_path, normalized_lines, fmt='%s', delimiter='\t')

# Ví dụ sử dụng hàm preprocess_text_file với một file có cột "comment"
preprocess_text_file(file_path1)